# Tasks

- We will work with Google BigQuery from Colab
- Some part of the instructions is coming from https://colab.research.google.com/notebooks/bigquery.ipynb
- We will use covid-19-public-data
- In particular, we use extract the UK data
  1. Check what kind of data are available
  2. Find out what we can do with the data



## Authentication

The first step is to 

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Load pacakges

In [2]:
from google.cloud import bigquery
import pandas as pd

## Connect to big query

Go to the console and check your project name and paste here

In [13]:
project_id = "gv918-ken"

In [12]:
client = bigquery.Client(project=project_id)

## Run queries

We will use `bigquery-public-data.covid19_open_data.covid19_open_data`

### Simple query

Get five rows from the table

- How many rows in the table?
- How many columns exist?

In [14]:
df = client.query('''
SELECT * 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
LIMIT 5
''').to_dataframe()

In [15]:
df

,date,location_key,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,cumulative_confirmed,cumulative_deceased,cumulative_recovered,cumulative_tested,new_hospitalized_patients,new_intensive_care_patients,new_ventilator_patients,cumulative_hospitalized_patients,cumulative_intensive_care_patients,cumulative_ventilator_patients,current_hospitalized_patients,current_intensive_care_patients,current_ventilator_patients,mobility_transit_stations,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_residential,mobility_workplaces,wikidata_id,datacommons_id,openstreetmap_id,latitude,longitude,location_geometry,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm
0,2020-09-25,US_NY_36059,US,United States of America,NY,New York,36059,Nassau County,US,USA,2,71,0,None,8013,46505,2201,None,813482,None,None,None,None,None,None,None,None,None,-42,-16,-7,107,10,-35,Q54064,geoId/36059,2552442,40.747222,-73.638056,POINT(-73.638056 40.747222),19.761111,15.272222,24.850000,0.0000,None
1,2020-09-26,US_NY_36059,US,United States of America,NY,New York,36059,Nassau County,US,USA,2,71,0,None,6097,46576,2201,None,819579,None,None,None,None,None,None,None,None,None,-23,-17,-6,176,2,-13,Q54064,geoId/36059,2552442,40.747222,-73.638056,POINT(-73.638056 40.747222),20.150000,15.866667,24.483333,0.0000,None
2,2020-09-27,US_NY_36059,US,United States of America,NY,New York,36059,Nassau County,US,USA,2,67,0,None,5350,46643,2201,None,824929,None,None,None,None,None,None,None,None,None,-31,-19,-14,119,3,-16,Q54064,geoId/36059,2552442,40.747222,-73.638056,POINT(-73.638056 40.747222),21.588889,17.744444,25.472222,0.0000,None
3,2020-09-28,US_NY_36059,US,United States of America,NY,New York,36059,Nassau County,US,USA,2,68,0,None,4544,46711,2201,None,829473,None,None,None,None,None,None,None,None,None,-50,-14,-10,109,13,-49,Q54064,geoId/36059,2552442,40.747222,-73.638056,POINT(-73.638056 40.747222),22.088889,19.677778,25.300000,1.0160,None
4,2020-09-29,US_NY_36059,US,United States of America,NY,New York,36059,Nassau County,US,USA,2,80,0,None,6359,46791,2201,None,835832,None,None,None,None,None,None,None,None,None,-45,-18,-11,40,13,-37,Q54064,geoId/36059,2552442,40.747222,-73.638056,POINT(-73.638056 40.747222),21.172222,19.538889,24.555556,0.8128,None


In [17]:
len(df.columns)

45

 - 5 Rows and 45 columns in the 'df' dataframe

### Explore the table

- How many countries in the data
- How many UK rows?
- What's the geographical units for the UK subnational levels

In [30]:
#How many countries in data
num_country = client.query('''
SELECT DISTINCT(country_code) 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
''').to_dataframe()
len(num_country)

247

In [38]:
#How many UK rows?
uk_rows = client.query('''
SELECT COUNT(*) 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE country_code = 'GB' 
''').to_dataframe()
uk_rows

,f0_
0,67768


In [23]:
df.columns

Index(['date', 'location_key', 'country_code', 'country_name',
       'subregion1_code', 'subregion1_name', 'subregion2_code',
       'subregion2_name', 'iso_3166_1_alpha_2', 'iso_3166_1_alpha_3',
       'aggregation_level', 'new_confirmed', 'new_deceased', 'new_recovered',
       'new_tested', 'cumulative_confirmed', 'cumulative_deceased',
       'cumulative_recovered', 'cumulative_tested',
       'new_hospitalized_patients', 'new_intensive_care_patients',
       'new_ventilator_patients', 'cumulative_hospitalized_patients',
       'cumulative_intensive_care_patients', 'cumulative_ventilator_patients',
       'current_hospitalized_patients', 'current_intensive_care_patients',
       'current_ventilator_patients', 'mobility_transit_stations',
       'mobility_retail_and_recreation', 'mobility_grocery_and_pharmacy',
       'mobility_parks', 'mobility_residential', 'mobility_workplaces',
       'wikidata_id', 'datacommons_id', 'openstreetmap_id', 'latitude',
       'longitude', 'location

- What's the latest date in the UK?

In [56]:
uk_rows = client.query('''
SELECT COUNT(*) 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE country_code = 'GB'
GROUP BY 
''').to_dataframe()
uk_rows

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



BadRequest: ignored

In [55]:
uk_subregion = client.query('''
SELECT DISTINCT(subregion2_name) 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data` 
WHERE country_code = 'GB'
''').to_dataframe()
uk_subregion

,subregion2_name
0,None
1,Hartlepool
2,Middlesbrough
3,Redcar and Cleveland
4,Stockton-on-Tees
...,...
178,Hywel Dda
179,Aneurin Bevan
180,Cardiff and Vale
181,Cwm Taf


In [48]:
client.query('''
SELECT subregion2_code, subregion2_name, COUNT(subregion2_name)
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE country_code = 'GB'
''')

- On 1 December 2020, at subregion2 level
  - Which sub region had the highest number of qumulative cases

In [54]:
uk_cases = client.query('''
SELECT *
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE county_code = 'GB' AND data = '2020-12-01' AND subregion2_code != 'None'
ORDER BY cumulative_confirmed DESC
''').to_dataframe()

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



BadRequest: ignored

- Mobility check on 1 Dec
  - How is the modiblity data look like?
    - `mobility_retail_and_recreation`
    - `mobility_parks`
    - `mobility_workplaces`

## Work with the Essex data

We can get the essex county data. Using it:

- Plot the case figure growth by date
- Plot mobility changes by date

Anything interesting?
